In [13]:
import os
import sys
sys.path.insert(0, '../gofher')

import itertools

from galaxy import galaxy
from spin_parity import read_spin_parity_galaxies_label_from_csv
from sparcfire import read_sparcfire_galaxy_csv, get_ref_band_and_gofher_params
from panstarrs import visualize_panstarrs, create_panstarrs_csv, PANSTARRS_REF_BANDS_IN_ORDER, PANSTARRS_BANDS_IN_ORDER
from sdss import visualize_sdss, create_sdss_csv, SDSS_REF_BANDS_IN_ORDER, SDSS_BANDS_IN_ORDER
#from run_gofher_on_a_galaxy import run_panstarrs_with_sparcfire_center_only, run_panstarrs_on_galaxy_with_sparcfire_center_inital_guess
from gofher import run_gofher_on_galaxy_with_fixed_center_only, run_gofher_on_galaxy_with_sparcfire_center_inital_guess
from file_helper import check_if_folder_exists_and_create

In [14]:
use_panstarrs = True

visualize_gals=True
make_csv=True

use_fixed_center = False #fixed center r, and use_fixed_center=True varying doen't change anything (since uses sep)

the_bulge_disk_r = 0.125
#the_bulge_disk_r = 0.25
#the_bulge_disk_r = 0.5
the_bulge_disk_string = str(the_bulge_disk_r).replace(".","")

In [15]:
if use_panstarrs:
    path_to_input = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\galaxies\\" #PANSTARRS
else:
    path_to_input = "C:\\Users\\school\\Desktop\\cross_id\\sdss_mosaic_construction" #SDSS

if use_panstarrs:
    if use_fixed_center:
        #output_folder_name = 'gofher_panstarrs_sparcfire_center_only_test' #PANSTARRS
        #output_folder_name = 'gofher_panstarrs_sparcfire_center_only_test_sans_g' #PANSTARRS
        output_folder_name = 'sparcfire_center' #PANSTARRS
    else:
        #output_folder_name = 'gofher_panstarrs_sparcfire_inital_guess' #PANSTARRS
        #output_folder_name = 'gofher_panstarrs_sparcfire_inital_guess_sans_g_r_{}'.format(the_bulge_disk_string) #PANSTARRS
        output_folder_name = 'inital_guess_{}'.format(the_bulge_disk_string) #PANSTARRS
else:
    if use_fixed_center:
        #output_folder_name = 'gofher_sdss_sparcfire_center_only_test' #SDSS
        output_folder_name = 'gofher_sdss_sans_u_sparcfire_center_only_test' #SDSS
    else:
        #output_folder_name = 'gofher_sdss_sparcfire_inital_guess' #SDSS
        output_folder_name = 'gofher_sdss_sans_u_sparcfire_inital_guess_r_{}'.format(the_bulge_disk_string) #SDSS

folder_name = "table3"
#path_to_output = "C:\\Users\\school\\Desktop\\gofher_output_refactor"
path_to_output = "E:\\grad_school\\research\\spin_parity_panstarrs"

def get_fits_path2(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_input,folder_name,name,"{}_{}.fits".format(name,band))

def get_galaxy_list():
    """the list of galaxy names to run on"""
    return os.listdir(os.path.join(path_to_input,folder_name))

def get_save_vis_path(name):
    """the file path specifying where to save the visualizitaion"""
    return os.path.join(path_to_output,output_folder_name,folder_name,"{}.png".format(name))

def get_csv_out_path():
    """the file path specifying where to save the ouput csv"""
    return os.path.join(path_to_output,output_folder_name,"{}.csv".format(folder_name))

def get_color_image_path(name):
    return os.path.join(path_to_input,folder_name,name,"{}_color.jfif".format(name))

def get_dark_side_csv_path():
    csv_path = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\table_info\\csv_format_of_table\\"
    return os.path.join(csv_path,"table_{}.csv".format(folder_name.strip()[-1]))

In [16]:
def setup_output_directories():
    output_path = os.path.join(path_to_output, output_folder_name)
    check_if_folder_exists_and_create(output_path)

    output_folder_path = os.path.join(path_to_output, output_folder_name, folder_name)
    check_if_folder_exists_and_create(output_folder_path)

In [17]:
def run_with_sparcfire_center_only(name, fits_path, sparcfire_bands, save_vis_path='', dark_side_label='', color_image_path='',bulge_disk_r=1.0):
    """run gofher on a single sdss galaxy"""
    the_gal = galaxy(name,dark_side_label)

    bands_in_oder = PANSTARRS_BANDS_IN_ORDER if use_panstarrs else SDSS_BANDS_IN_ORDER
    ref_bands_in_order = PANSTARRS_REF_BANDS_IN_ORDER if use_panstarrs else SDSS_REF_BANDS_IN_ORDER

    for band in bands_in_oder:
        the_gal.construct_band(band,fits_path(name,band))

    the_ref_band, the_sparcfire_derived_params = get_ref_band_and_gofher_params(sparcfire_bands,ref_bands_in_order,bulge_disk_r)
    #print(the_sparcfire_derived_params.a,the_sparcfire_derived_params.b)
    if the_ref_band == None or the_sparcfire_derived_params == None: return

    the_gal.ref_band = the_ref_band
    the_band_pairs = list(itertools.combinations(bands_in_oder, 2))

    the_gal = run_gofher_on_galaxy_with_fixed_center_only(the_gal,the_band_pairs,the_sparcfire_derived_params)
    if save_vis_path != '':
        if use_panstarrs:
            visualize_panstarrs(the_gal,save_vis_path,color_image_path)
        else:
            visualize_sdss(the_gal,save_vis_path)
    return the_gal

def run_with_sparcfire_center_inital_guess(name, fits_path, sparcfire_bands, save_vis_path='', dark_side_label='', color_image_path='',bulge_disk_r=1.0):
    """run gofher on a single sdss galaxy"""
    the_gal = galaxy(name,dark_side_label)

    bands_in_oder = PANSTARRS_BANDS_IN_ORDER if use_panstarrs else SDSS_BANDS_IN_ORDER
    ref_bands_in_order = PANSTARRS_REF_BANDS_IN_ORDER if use_panstarrs else SDSS_REF_BANDS_IN_ORDER

    for band in bands_in_oder:
        the_gal.construct_band(band,fits_path(name,band))

    the_ref_band, the_sparcfire_derived_params = get_ref_band_and_gofher_params(sparcfire_bands,ref_bands_in_order,bulge_disk_r)
    #print(the_sparcfire_derived_params.a,the_sparcfire_derived_params.b)
    if the_ref_band == None or the_sparcfire_derived_params == None: return

    the_gal.ref_band = the_ref_band
    the_band_pairs = list(itertools.combinations(bands_in_oder, 2))

    the_gal = run_gofher_on_galaxy_with_sparcfire_center_inital_guess(the_gal,the_band_pairs,the_sparcfire_derived_params)
    if save_vis_path != '':
        if use_panstarrs:
            visualize_panstarrs(the_gal,save_vis_path,color_image_path)
        else:
            visualize_sdss(the_gal,save_vis_path)
    return the_gal

In [18]:
if use_panstarrs:
    path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\output_from_running\\SpArcFiRe\\{}\\galaxy.csv".format(folder_name) #PANSTARRS
else:
    path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\cross_id\\sdss_mosaic_construction\\SpArcFiRe_output\\{}\\G.out\\galaxy.csv".format(folder_name) #SDSS
sparcfire_galaxy_csv = read_sparcfire_galaxy_csv(path_to_sparcfire_csv)

dark_side_labels = read_spin_parity_galaxies_label_from_csv(get_dark_side_csv_path())
if folder_name == "table2" and "IC 2101" in dark_side_labels:
    dark_side_labels["IC2101"] = dark_side_labels["IC 2101"]

setup_output_directories()

the_gals = []
i = 1
for name in sparcfire_galaxy_csv:
    print(i,name)
    i += 1
    try:
        sparcfire_bands = sparcfire_galaxy_csv[name]

        save_vis_path = ''
        if visualize_gals:
            save_vis_path=get_save_vis_path(name)

        paper_dark_side_label = dark_side_labels[name]

        if use_fixed_center:
            the_gal = run_with_sparcfire_center_only(name, get_fits_path2, sparcfire_bands, save_vis_path=save_vis_path,dark_side_label=paper_dark_side_label,color_image_path=get_color_image_path(name),bulge_disk_r=the_bulge_disk_r)
        else:
            the_gal = run_with_sparcfire_center_inital_guess(name, get_fits_path2, sparcfire_bands, save_vis_path=save_vis_path,dark_side_label=paper_dark_side_label,color_image_path=get_color_image_path(name),bulge_disk_r=the_bulge_disk_r)
        the_gals.append(the_gal)
        #break
    except Exception as e:
        print("Error when running on",name,e)
        #break

if make_csv:
    if use_panstarrs:
        the_band_pairs = list(itertools.combinations(PANSTARRS_BANDS_IN_ORDER, 2)) #PANSTARRS
        create_panstarrs_csv(the_gals,the_band_pairs,get_csv_out_path())
    else:
        the_band_pairs = list(itertools.combinations(SDSS_BANDS_IN_ORDER, 2)) #SDSS
        create_sdss_csv(the_gals,the_band_pairs,get_csv_out_path())

1 IC1151
2 IC1199
3 IC1256
4 IC1528
5 IC2095
6 IC2163
7 IC2487
8 IC4566
Error when running on IC4566 Band:r with fits_path:C:\Users\school\Desktop\github\spin-parity-catalog\original\galaxies\table3\IC4566\IC4566_r.fits is invalid
9 IC467
10 IC5309
11 IC674
12 IC776
13 NGC1012
14 NGC1042
15 NGC1058
16 NGC1068
17 NGC1073
18 NGC1358
19 NGC1614
20 NGC1645
21 NGC1677
22 NGC171
23 NGC177
24 NGC180
25 NGC1832
26 NGC192
27 NGC1
28 NGC2139
Error fitting sersic, using inital_gofher_parameters from sep
29 NGC214
30 NGC217
31 NGC2207
32 NGC234
33 NGC237
34 NGC23
35 NGC2449
36 NGC2486
37 NGC2525
38 NGC2540
39 NGC257
40 NGC2604
Error fitting sersic, using inital_gofher_parameters from sep
41 NGC2608
42 NGC2730
43 NGC2776
44 NGC278
45 NGC289
Error when running on NGC289 Band:r with fits_path:C:\Users\school\Desktop\github\spin-parity-catalog\original\galaxies\table3\NGC289\NGC289_r.fits is invalid
46 NGC2906
47 NGC2916
48 NGC2964
49 NGC2998
50 NGC3106
51 NGC3145
52 NGC3166
53 NGC3614
54 NGC3631
55 N